<a href="https://colab.research.google.com/github/softmurata/colab_notebooks/blob/main/video/tune_a_video.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# download stable-diffusion-v1-4 checkpoints
%cd /content
!git lfs install
!git clone https://huggingface.co/CompVis/stable-diffusion-v1-4

In [ ]:
!git clone https://github.com/showlab/Tune-A-Video.git

In [ ]:
# Tips: torch, torchvisionはcomment out
%cd /content/Tune-A-Video/
!pip install -r requirements.txt
!pip install -U xformers

In [ ]:
%cd /content/Tune-A-Video/

In [9]:
!accelerate launch train_tuneavideo.py --config="configs/man-surfing.yaml"

	`--num_processes` was set to a value of `1`
	`--num_machines` was set to a value of `1`
	`--mixed_precision` was set to a value of `'no'`
	`--dynamo_backend` was set to a value of `'no'`
To avoid this warning pass in values for each of the problematic parameters or run `accelerate config`.
Error caught was: No module named 'triton'
INFO:__main__:Distributed environment: NO
Num processes: 1
Process index: 0
Local process index: 0
Device: cuda
Mixed precision type: fp16

{'variance_type', 'prediction_type'} was not found in config. Values will be initialized to default values.
{'norm_num_groups'} was not found in config. Values will be initialized to default values.
{'use_linear_projection', 'resnet_time_scale_shift', 'num_class_embeds', 'only_cross_attention', 'dual_cross_attention', 'class_embed_type', 'mid_block_type', 'upcast_attention'} was not found in config. Values will be initialized to default values.
{'prediction_type'} was not found in config. Values will be initialized to d

In [11]:
!pip install triton

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 76.0 MB/s eta 0:00:00


In [12]:
#@title inference
from tuneavideo.pipelines.pipeline_tuneavideo import TuneAVideoPipeline
from tuneavideo.models.unet import UNet3DConditionModel
from tuneavideo.util import save_videos_grid
import torch

pretrained_model_path = "/content/stable-diffusion-v1-4"
unet_model_path = "./outputs/man-surfing_lr3e-5_seed33/2023-02-08T01-11-37"
unet = UNet3DConditionModel.from_pretrained(unet_model_path, subfolder='unet', torch_dtype=torch.float16).to('cuda')
pipe = TuneAVideoPipeline.from_pretrained(pretrained_model_path, unet=unet, torch_dtype=torch.float16).to("cuda")
pipe.enable_xformers_memory_efficient_attention()

prompt = "a panda is surfing"
video = pipe(prompt, video_length=8, height=512, width=512, num_inference_steps=50, guidance_scale=7.5).videos

save_videos_grid(video, f"./{prompt}.gif")

  0%|          | 0/50 [00:00<?, ?it/s]